# Handling Missing Data

### Trade-Offs in Missing Data Conventions

The two main options are:

* Masks

* Sentinal Values

### How Pandas Handles Missing Data

R uses specific bit patters to indicate nullness

Numpy uses masks

### NumPy has a hard time with *None* but handles *NaN* well

In [1]:
import numpy as np
import pandas as pd

In [4]:
# Using the 'None' value for missing data in an array identifies the dtype as 'object'
# This is the only common dtype for any element that can be stored in the array
# 
# With the datatype as 'object' it will lead to inefficiency since operations will be
# done at the Python level and not take advantage of the bitwise operations of NumPy

vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [7]:
# Trying to apply aggregation methods to the array will yield errors.

vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### NaN : Missing Numerical Data

In [12]:
# Using the 'np.nan' method returns the 'NaN' value for that element.
# This allows NumPy to treat it as a null value and it is ignored in operations.

vals2 = np.array([1, np.nan, 3, 4])
print(vals2.dtype)
print(vals2)


float64
[ 1. nan  3.  4.]


In [16]:
# Operations including an  'NaN' return 'NaN'
# This can mess things up when attempting to aggregate data 
# with a missing values

[1 + vals2], vals2.sum(), vals2.min(), vals2.max()

([array([ 2., nan,  4.,  5.])], nan, nan, nan)

In [18]:
# NumPy provides aggregations with the '.nan___' prefix to ignore the null values

np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

### Missing Data in Pandas : NaN and None

In [20]:
# NaN and None both have their place in Pandas and converts them where appropriate

pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [26]:
# When an NaN is put into a pandas object the dtype is automatically converted to a float
# 'None' values are also converted in 'NaN'
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [27]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

In [28]:
'''
Typeclass Conversion When Storing NAs
NA                      Sentinel Value
---------------------------------------
floating----> No change ----------> np.nan
object  ----> No change ----------> None or np.nan
integer ----> Cast to float64 ----> np.nan
boolean ----> Cast to object -----> None or np.nan
''';

### Operating on Null Values

In [29]:
'''
Here are some useful methods for detecting and working with null values in Pandas:

    isnull(): Generate a boolean mask indicating missing values
    notnull(): Opposite of isnull()
    dropna(): Return a filtered version of the data
    fillna(): Return a copy of the data with missing values filled or imputed

''';

In [32]:
# You can detect null values using the '.isnull()' and '.notnull()' methods

data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [34]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [35]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

In [37]:
# '.isnull' apply directly to the DataFrame

data[data.notnull()]


0        1
2    hello
dtype: object

### Dropping Null Values from a DataFrame

In [38]:
# '.dropna()' doesn't modify the Series

data.dropna()

0        1
2    hello
dtype: object

In [39]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [41]:
# When operating on a DataFrame '.dropna()' effects entire rows or columns

df = pd.DataFrame([[1, np.nan, 2],
                  [2,      3, 5],
                  [np.nan, 4, 6]])

df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [42]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [48]:
# For Reference
# For the axis keyword
#
# using the 'columns' string is equivalent to saying axis=1
#
# using the 'rows' string is equivalent to saying axis=0

df.dropna(axis=1)

,2
0,2
1,5
2,6


In [49]:
df.dropna(axis='rows')

,0,1,2
1,2.0,3.0,5


In [50]:
# There is a keyword of 'how'
# It defaults to 'any' so that any row or column with ANY 'NaN' values gets dropped.
# You can specify 'how = 'all'' so that every element of the row or column
# must be 'NaN' in order for it to be dropped.

df.dropna(axis='rows', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [51]:
# if we add a column of 'NaN' values this command will work

df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [52]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [53]:
# The 'thresh' keyword specifies the number of non-null values to keep the column or row.

df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [54]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Filling null values

In [63]:
data = pd.Series([1, np.nan, np.nan, 2, None, 3], index=list('abcdef'))
data

a    1.0
b    NaN
c    NaN
d    2.0
e    NaN
f    3.0
dtype: float64

In [64]:
# '[pandas object].fillna(____)' fills the pandas object with whatever value is input.

data.fillna('turkey')

a       1.0
b    turkey
c    turkey
d       2.0
e    turkey
f       3.0
dtype: object

In [65]:
# Using the 'method' keyword you can specify various ways to fill

print(data.fillna(method='ffill'), '\n') # fills with the value of the previous element

print(data.fillna(method='bfill'))       # fills with the value of the subsequent element



a    1.0
b    1.0
c    1.0
d    2.0
e    2.0
f    3.0
dtype: float64 

a    1.0
b    2.0
c    2.0
d    2.0
e    3.0
f    3.0
dtype: float64


In [78]:
# When using '.fillna(_____)' with DataFrames the axis can also be specified

df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [80]:
# With the 'ffill' method if the first encountered element is null it will stay null
# This works similarly with 'bfill' if the last element is null.
df.fillna(method='ffill', axis=1) 

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [83]:
df.fillna(method='bfill')

,0,1,2,3
0,1.0,3.0,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN
